<a href="https://colab.research.google.com/github/pathakdevanshu/project1/blob/main/MNIST_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import cv2
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as f
import torchvision.datasets as dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transform

# **Preparing the Dataset**

In [64]:
dataset=pd.read_csv("/content/sample_data/mnist_train_small.csv")
print(dataset.shape)
df=np.array(dataset)
x=df[:,1:]
y=df[:,0]
x=torch.tensor(x,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.int64)

(19999, 785)


In [65]:
datasett=[]
for i in range(len(df)):
  xv,yv=x[i],y[i]
  xv=xv.reshape(1,28,28)
  datasett.append((xv,yv))

# **Importing in Dataloader:--**

In [66]:
data=DataLoader(datasett,batch_size=10,shuffle=True)

## **CNN Model:---**

In [69]:
class cnn(nn.Module):
  def __init__(self):
    super().__init__()
    self.cn1=nn.Conv2d(1,16,5,1)
    self.cn2=nn.Conv2d(16,32,5,1)
    self.r=nn.ReLU(inplace=True)
    self.pool=nn.MaxPool2d(2)
    self.fc1=nn.Linear(32*4*4,100)
    self.fc2=nn.Linear(100,10)
  def forward(self,x):
    x=self.pool(self.r(self.cn1(x)))
    x=self.pool(self.r(self.cn2(x)))
    x=x.reshape(-1,32*16)
    x=f.relu(self.fc1(x))
    return self.fc2(x)
model=cnn()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

## **Training the Model:-**

In [70]:
epoch=15
for i in range(epoch):
  tloss=0
  for j,(x,y) in enumerate(data):
    if j==1800:
      break
    yp=model.forward(x)
    loss=criterion(yp,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    tloss=tloss+loss
  print("TOtal lOss = ",tloss)

TOtal lOss =  tensor(442.5659, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(190.3549, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(141.3521, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(129.4188, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(116.6020, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(104.3477, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(100.8088, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(85.9618, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(115.5169, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(82.4170, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(81.2837, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(83.2561, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(67.2733, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(98.3131, grad_fn=<AddBackward0>)
TOtal lOss =  tensor(83.7013, grad_fn=<AddBackward0>)


**Test Data:-**

In [99]:
testdata=pd.read_csv("/content/sample_data/mnist_test.csv")
df=np.array(testdata)
xt=df[:,1:]
yt=df[:,0]
xt=torch.tensor(xt,dtype=torch.float32)
yt=torch.tensor(yt,dtype=torch.int64)

In [100]:
xt=xt.reshape(-1,1,1,28,28)

In [102]:
ypred,yreal=[],[]
for i in range(len(testdata)):
  yp=model.forward(xt[i])
  yp=torch.softmax(yp,dim=1)
  yp=torch.argmax(yp,dim=1)
  ypred.append(yp)
  yreal.append(yt[i])

# **Confusion Matrix:--**

In [108]:
from sklearn.metrics import confusion_matrix
confusion_matrix(yreal,ypred)

array([[ 971,    0,    1,    1,    3,    1,    1,    0,    0,    2],
       [   0, 1122,    6,    2,    1,    0,    0,    4,    0,    0],
       [   5,    0, 1009,    6,    2,    0,    0,   10,    0,    0],
       [   0,    1,    3,  991,    0,    8,    0,    4,    2,    1],
       [   1,    0,    0,    0,  970,    0,    1,    1,    0,    9],
       [   2,    0,    0,   13,    0,  869,    2,    1,    0,    5],
       [   6,    5,    2,    1,    5,    4,  935,    0,    0,    0],
       [   0,    2,    8,    2,    0,    0,    0, 1010,    0,    5],
       [   8,    1,   15,   11,    3,    2,    1,    3,  919,   11],
       [   2,    0,    0,    1,    9,    5,    0,    9,    1,  982]])

In [110]:
#Accuracy:--
c=0
for i in range(9999):
  if ypred[i]!=yreal[i]:
    c=c+1
print("ACCURACY:-",((9999-c)/9999)*100)

ACCURACY:- 97.7897789778978
